In [1]:
from ase.io import read
from ase.visualize import view

# Load the XDATCAR file
file_path = "XDATCAR_mlff_1h2o2alafi_1_istart0"  # Replace with your XDATCAR file path
atoms = read(file_path, index=':')  # Load all frames (trajectory)

# Visualize the structure
view(atoms)  # Opens an interactive viewer

<Popen: returncode: None args: ['/home/kianoush/Desktop/kianoush/test/bin/py...>

convert it to a function

In [9]:
file_path = "XDATCAR_mlff_1h2o2alafi_1_istart0"  # Replace with the actual file path

try:
    data = []
    with open(file_path, 'r') as file:
        inside_coords = False
        curr_coords = []
        for line in file:
            if line.startswith("Direct configuration"):
                inside_coords = True
                curr_coords = []
            elif inside_coords:
                row = line.split()
                float_row = [float(element) for element in row]
                curr_coords.append(float_row)
                if len(curr_coords) == 77:
                    data.append(curr_coords)
                    inside_coords = False
except FileNotFoundError:
    print("File not found.")
except IOError:
    print("Error reading the file.")

# for set_coords in data:
#     for row in set_coords:
#         print(row)

In [10]:
data[2]

[[0.1911823, 0.18763797, 0.89045737],
 [0.31834707, 0.03313912, 0.11891085],
 [0.17176566, 0.28665963, 0.95329643],
 [0.26072845, 0.29411292, 0.14707305],
 [0.5499564, 0.8599614, 0.49993664],
 [0.09981584, 0.66989796, 0.51492355],
 [0.32004272, 0.410034, 0.51734475],
 [0.33993655, 0.85010022, 0.00068816],
 [0.50928562, 0.63948895, 0.00089934],
 [0.47608246, 0.32330758, 0.01684394],
 [0.00015121, 0.63036894, 0.79664592],
 [0.64025648, 0.00033016, 0.74974498],
 [0.40042453, 0.20789296, 0.28241814],
 [0.64132795, 0.55256743, 0.51401792],
 [0.83970871, 0.54028245, 0.99995239],
 [0.88992965, 0.35979734, 0.50031954],
 [0.64273221, 0.64409926, 0.78152678],
 [0.3388411, 0.35063574, 0.19751137],
 [0.46991919, 0.10009782, 0.51553457],
 [0.01003408, 0.369971, 0.74999271],
 [0.37006922, 0.00991951, 0.79679004],
 [0.42027867, 0.85997748, 0.71904428],
 [0.97991591, 0.37993462, 0.21878389],
 [0.37281352, 0.37396899, 0.82341493],
 [0.6301457, 0.6400572, 0.2343938],
 [0.9899565, 0.62997331, 0.23411256]

In [11]:
# Converting Fractional coordinates to Real Coordinates
file_path = "XDATCAR_mlff_1h2o2alafi_1_istart0"   # Replace with the actual input file path
output_file_path = "all_converted_coordinates.txt"  # Replace with the desired output file path

try:
    data = []
    with open(file_path, 'r') as input_file:
        inside_coords = False
        curr_coords = []
        for line in input_file:
            if line.startswith("Direct configuration"):
                inside_coords = True
                curr_coords = []
                set_number = line.split("=")[-1].strip()  # Extract the set number
            elif inside_coords:
                row = line.split()
                float_row = [float(element) for element in row]
                curr_coords.append(float_row)
                if len(curr_coords) == 77:
                    data.append((set_number, curr_coords))
                    inside_coords = False

    if data:
        with open(output_file_path, 'w') as output_file:
            for set_data in data:
                set_number, set_coords = set_data
                output_file.write(f"Direct configuration= {set_number}\n")  # Write the set label
                for row in set_coords:
                    converted_row = [
                        row[0] * 13.677735,
                        row[1] * 11.845266,
                        row[2] * 8.602417
                    ]
                    converted_row_str = [format(element, ".8f") for element in converted_row]
                    output_file.write(" ".join(converted_row_str) + "\n")
                output_file.write("\n")  # Add an empty line between sets
except FileNotFoundError:
    print("File not found.")
except IOError:
    print("Error reading/writing the file.")

In [12]:
# just to show example of the real coordinates

file_path = "all_converted_coordinates.txt"  # Replace with the actual file path

try:
    with open(file_path, 'r') as file:
        # Read and print the first 10 lines
        for i in range(10):
            line = file.readline()
            if not line:  # Stop if there are fewer than 10 lines
                break
            print(line.strip())  # Print line without trailing newline
except FileNotFoundError:
    print("File not found.")
except IOError:
    print("Error reading the file.")

Direct configuration= 1
2.58116394 2.28487944 7.70000462
4.34921102 0.43199448 1.04474144
2.32419268 3.37488733 8.18053013
3.58140330 3.51941936 1.20093664
7.52404187 10.18724811 4.29977104
1.36672414 7.93737060 4.43068485
4.37864072 4.85770331 4.45221103
4.64963276 10.07120632 0.00723317
6.97077612 7.57769573 0.00572568


In [13]:
# By this ode we will add a new column to the angles file so we will determine what is the central atom in the angles



# Define a function to process the file
def process_file(input_file, output_file):
    # Define ranges for atom types
    ranges = {
        "Hydrogen": range(0, 4),  # Includes 0, 1, 2, 3
        "Oxygen": range(4, 53),   # Includes 4 to 52
        "Al": range(53, 55),      # Includes 53, 54
        "Si": range(55, 77),      # Includes 55 to 76
    }

    # Open the input file and prepare to write to the output file
    with open(input_file, "r") as infile, open(output_file, "w") as outfile:
        for line in infile:
            # Ignore empty lines
            if not line.strip():
                continue

            # Parse the line
            parts = list(map(int, line.split()))
            if len(parts) != 3:
                raise ValueError(f"Invalid line format: {line.strip()}")

            # Extract the central number (second number)
            central = parts[1]

            # Determine the atom type based on the central number
            atom = next((key for key, rng in ranges.items() if central in rng), "Unknown")

            # Write the updated line to the output file
            outfile.write(f"{parts[0]} {parts[1]} {parts[2]} {atom}\n")

# Specify input and output file paths
input_file = "Output_Angles.txt"  # Replace with your input file path
output_file = "Modified_Output_Angles.txt"  # Replace with your desired output file path

# Call the function to process the file
process_file(input_file, output_file)


In [14]:
# In this code we count how many angles have a central atoms of each element (it should be 144 Si or Al, and 53 (1 water) && (4 angles for H-O-Al or H-O-Si)
# && (48 angles for Si-O-Si and Si-O-Al) && (144 angles with Si or Al as central atoms O-Si_O or O-Si-Al).


# Define a function to count atom types in the last column
def count_atoms(file_path):
    # Initialize counters
    atom_counts = {
        "Hydrogen": 0,
        "Oxygen": 0,
        "Al": 0,
        "Si": 0,
    }

    # Read the file and count atom types
    with open(file_path, "r") as infile:
        for line in infile:
            # Ignore empty lines
            if not line.strip():
                continue

            # Get the last column (atom type)
            parts = line.split()
            atom = parts[-1]  # Last column contains the atom type

            # Increment the corresponding counter if the atom is known
            if atom in atom_counts:
                atom_counts[atom] += 1

    return atom_counts

# Specify the file path
file_path = "Modified_Output_Angles.txt"  # Replace with the path to your processed file

# Get the atom counts
counts = count_atoms(file_path)

# Print the results
print("Atom Counts:")
for atom, count in counts.items():
    print(f"{atom}: {count}")

Atom Counts:
Hydrogen: 0
Oxygen: 53
Al: 12
Si: 132


In [16]:
# To modify the code to add the labels and ensure that the columns are well separated, you can replace the section where the new columns are added and calculated. Here's the modified code:


import math

file_path = "all_converted_coordinates.txt"  # Replace with the correct input file path
output_file_path = "modify_all_converted_coordinates.txt"  # Replace with the desired output file path


# Read the bond connections from bond.txt
bond_connections = []            # creates a list of bonds
with open("bond.txt", "r") as file:        # This opens a file named bond.txt in read mode ("r"). The with statement ensures that the file is properly opened and closed after reading. 
    for line in file:
        atoms = line.split()   # For each line in the file, this splits the line into a list of words or values, using whitespace as the delimiter. The result is stored in the atoms list.
        if len(atoms) == 2:   
            bond_connections.append((int(atoms[0]), int(atoms[1]))) # # Creates a tuple containing these two integers 
            # (representing a bond between the two atoms).
# Appends this tuple to the bond_connections list.


# Read the bond connections from bond.txt
angle_connections = []
with open("Output_Angles.txt", "r") as file:
    for line in file:
        atoms = line.split()
        if len(atoms) == 3:
            angle_connections.append((int(atoms[0]), int(atoms[1]), int(atoms[2])))

try:
    data = []
    with open(file_path, 'r') as input_file:
        inside_coords = False
        curr_coords = []
        for line in input_file:
            if line.startswith("Direct configuration"):
                inside_coords = True
                curr_coords = []
                set_number = line.split("=")[-1].strip()  # Extract the set number
            elif inside_coords:
                row = line.split()
                float_row = [float(element) for element in row]
                curr_coords.append(float_row)
                if len(curr_coords) == 77:
                    data.append((set_number, curr_coords))
                    inside_coords = False
    
    if data:
        with open(output_file_path, 'w') as output_file:
            for set_data in data:
                set_number, set_coords = set_data
                output_file.write(f"Direct configuration= {set_number}\n")  # Write the set label
                
                # Calculating bond differences for each set of coordinates
                bond_diff_lines = []
                for atom1, atom2 in bond_connections:
                    coords1 = set_coords[atom1]
                    coords2 = set_coords[atom2]
                    diff_x = coords1[0] - coords2[0]
                    diff_y = coords1[1] - coords2[1]
                    diff_z = coords1[2] - coords2[2]
                    distance_diff = math.sqrt(diff_x**2 + diff_y**2 + diff_z**2)
                    line = f"{atom1} {atom2} {diff_x:.6f} {diff_y:.6f} {diff_z:.6f} {distance_diff:.6f}\n"
                    bond_diff_lines.append(line)
                
                # Perform calculations and append new columns
                new_column_6 = []
                new_column_7 = []
                new_column_8 = []
                new_column_9 = []  # Initialize new column for distance
                
                a = 13.677735
                b = 11.845266
                c = 8.602417
                for line in bond_diff_lines:
                    data = line.split()
                    column_1 = data[0]
                    column_2 = data[1]
                    column_3 = float(data[2])
                    column_4 = float(data[3])
                    column_5 = float(data[4])
                    
                    # Check if absolute value is greater than 0.5 * a
                    if abs(column_3) > 0.5 * a:
                        # Subtract a if value is greater than 0, otherwise add a
                        if column_3 > 0:
                            new_column_6.append(column_3 - a)
                        else:
                            new_column_6.append(column_3 + a)
                    else:
                        new_column_6.append(column_3)

                    # Check if absolute value is greater than 0.5 * b
                    if abs(column_4) > 0.5 * b:
                        # Subtract b if value is greater than 0, otherwise add b
                        if column_4 > 0:
                            new_column_7.append(column_4 - b)
                        else:
                            new_column_7.append(column_4 + b)
                    else:
                        new_column_7.append(column_4)

                    # Check if absolute value is greater than 0.5 * c
                    if abs(column_5) > 0.5 * c:
                        # Subtract c if value is greater than 0, otherwise add c
                        if column_5 > 0:
                            new_column_8.append(column_5 - c)
                        else:
                            new_column_8.append(column_5 + c)
                    else:
                        new_column_8.append(column_5)
                    
                    # Calculate the distance using columns 6, 7, and 8
                    distance_diff = math.sqrt(new_column_6[-1]**2 + new_column_7[-1]**2 + new_column_8[-1]**2)
                    new_column_9.append(distance_diff)
                
                # Write the column labels
                output_file.write("atom_1     atom_2     diff_x            diff_y            diff_z            distance          modified_x_distance  modified_y_distance  modified_z_distance  real_distance\n")
                
                # Update the bond difference lines with new columns and write to the output file
                for i in range(len(bond_diff_lines)):
                    output_file.write("{:<10}{:<10}{:<18.6f}{:<18.6f}{:<18.6f}{:<18.6f}{:<20.6f}{:<20.6f}{:<20.6f}{:<20.6f}\n".format(
                        bond_connections[i][0], bond_connections[i][1], float(bond_diff_lines[i].split()[2]),
                        float(bond_diff_lines[i].split()[3]), float(bond_diff_lines[i].split()[4]), float(bond_diff_lines[i].split()[5]), 
                        new_column_6[i], new_column_7[i], new_column_8[i], new_column_9[i]
                    ))
                
                output_file.write("\n")  # Add an empty line between sets
except FileNotFoundError:
    print("File not found.")
except IOError:
    print("Error reading/writing the file.")


# In this modified code, the column labels are added as a separate line after writing the set label. The `output_file.write` 
# function is used to write each column name, and the `"{:<10}{:<10}{:<18.6f}{:<18.6f}..."` format string is used to 
# ensure the columns are well separated. The column values are extracted from the `bond_diff_lines` list to ensure consistency.

In [17]:
# Angle between two vectors
import math

def find_angle(v1, v2):
    dot_product = sum(a * b for a, b in zip(v1, v2))
    magnitude_v1 = math.sqrt(sum(a**2 for a in v1))
    magnitude_v2 = math.sqrt(sum(b**2 for b in v2))
    theta = math.acos(dot_product / (magnitude_v1 * magnitude_v2))
    
    return math.degrees(theta)

# Example usage
v1 = [1, 2, 3]  # Coordinates of the first vector
v2 = [4, 5, 6]  # Coordinates of the second vector
angle = find_angle(v1, v2)
print(f"The angle between v1 and v2 is: {angle} degrees")

The angle between v1 and v2 is: 12.933154491899135 degrees


### The following reads the rows in the Angles.txt file which are atom labels involving in each Angle and stores each row's label in a column (column1 2 & 3) we later use it to identify the pair of rows which we compute angle betrween them: see the section # for the whole dataset 

In [18]:
# Initialize empty lists to store each column
column1 = []
column2 = []
column3 = []

# Read the text file with three columns
with open('Output_Angles.txt', 'r') as file:
    for line in file:
        # Split the line into columns based on whitespace
        columns = line.split()
        if len(columns) == 3:
            # Store each column in the respective list
            column1.append(columns[0])
            column2.append(columns[1])
            column3.append(columns[2])
        else:
            print("Error: Line does not contain exactly three columns")

# Print or use the lists as needed
print("Column 1:", column1)
print("Column 2:", column2)
print("Column 3:", column3)

# Print the length of each column
print("Length of Column 1:", len(column1))
print("Length of Column 2:", len(column2))
print("Length of Column 3:", len(column3))

Error: Line does not contain exactly three columns
Column 1: ['67', '55', '53', '64', '65', '56', '55', '57', '54', '63', '58', '60', '63', '53', '54', '60', '59', '64', '69', '56', '65', '68', '67', '54', '62', '68', '57', '71', '66', '58', '57', '56', '55', '74', '53', '68', '65', '66', '54', '61', '53', '72', '58', '57', '55', '61', '56', '67', '38', '6', '6', '17', '17', '6', '27', '12', '12', '18', '18', '12', '36', '5', '5', '10', '10', '5', '35', '9', '9', '23', '23', '9', '34', '30', '30', '11', '11', '11', '33', '10', '10', '14', '14', '10', '35', '18', '18', '20', '20', '18', '34', '15', '15', '19', '19', '15', '43', '6', '6', '23', '23', '6', '43', '19', '19', '28', '28', '19', '13', '16', '13', '33', '13', '16', '7', '7', '20', '7', '21', '20', '8', '8', '40', '8', '24', '24', '13', '13', '24', '13', '32', '24', '4', '4', '40', '4', '26', '26', '5', '5', '25', '5', '29', '25', '22', '22', '28', '22', '29', '28', '7', '7', '39', '7', '27', '27', '14', '14', '25', '14', '25',

I have two lists, a and b, also I have a text file named new.txt. the new.txt has three columns itself. 
a and b contain numbers that specify the row number. for all a and b's subtract the first element of the a's row in the new.txt file from the first element of the b's row in the new.txt file. and for all a and b's subtract the second element of the a's row in the new.txt file from the element column of the b's row in the new.txt file. and for all a and b's subtract the third element of the a's row in the new.txt file from the third element of the b's row in the new.txt file. 

In [42]:
with open('new.txt', 'r') as file:
    data = [list(map(float, line.split())) for line in file]

# Define the functions to subtract two rows
def subtract_rows(row1, row2):
    return [row2[i] - row1[i] for i in range(len(row1))]

# Initialize a list to store the results
results = []

# Loop through the rows specified in lists a and b
for i in range(len(column1)):
    row_column1 = data[int(column1[i])] # Subtract 1 because lists are zero-indexed
    row_column2 = data[int(column2[i])] # Subtract 1 because lists are zero-indexed
    result = subtract_rows(row_column1, row_column2)
    results.append(result)

# Print the results
for result in results:
    print(result)


# Write the results to a new text file
with open('1_2.txt', 'w') as file:
    for result in results:
        file.write(' '.join(map(str, result)) + '\n')

print("Results have been written to output.txt file.")

[-0.017643050000000216, -0.3119853499999987, 1.5927106700000002]
[-0.2518668799999999, -0.0308667499999995, -1.5926693800000002]
[0.13869072999999954, -0.4435644700000001, 1.56449216]
[0.11714870000000044, -0.31868419999999986, -7.02733862]
[-0.3075585099999998, -0.32066367000000007, -1.5609378999999999]
[0.043921250000000356, -0.1602331699999997, -7.0115903500000005]
[-1.61608471, -0.4995185399999995, 0.8287346600000003]
[-0.5722656199999996, -1.47162232, -0.5578136599999999]
[-0.8331494099999999, 0.91728152, -0.41172432999999975]
[-0.4433663800000005, -0.031224829999999315, -1.53706851]
[-0.42725153999999854, -0.06160213999999975, 1.551987770000001]
[0.0730697400000011, 0.24266580000000015, -1.6169069499999997]
[-0.4236299500000005, 1.0628817600000007, 0.7847404400000002]
[0.38434161999999983, -1.15377155, -1.17394226]
[0.11697472999999992, -0.35306166000000005, 1.5934909899999998]
[-11.96262718, 0.3601272400000006, 0.5293900699999998]
[-1.3166059099999998, -1.31619621, 0.85227422999

In [43]:
with open('new.txt', 'r') as file:
    data = [list(map(float, line.split())) for line in file]

# Define the functions to subtract two rows
def subtract_rows(row1, row2):
    return [row2[i] - row1[i] for i in range(len(row1))]

# Initialize a list to store the results
results = []

# Loop through the rows specified in lists a and b
for i in range(len(column1)):
    row_column2 = data[int(column2[i])] # Subtract 1 because lists are zero-indexed
    row_column3 = data[int(column3[i])] # Subtract 1 because lists are zero-indexed
    result = subtract_rows(row_column2, row_column3)
    results.append(result)

# Print the results
for result in results:
    print(result)


# Write the results to a new text file
with open('3_2.txt', 'w') as file:
    for result in results:
        file.write(' '.join(map(str, result)) + '\n')

print("Results have been written to output.txt file.")

[-0.11027645999999969, 0.14914326999999972, 1.5852294899999997]
[0.16256329999999997, -0.046240720000000124, -1.6530716699999997]
[0.18747400999999986, 0.3923740800000006, 1.5638091200000002]
[-0.2284868400000004, 0.22725390999999995, 1.56192021]
[0.3253395699999997, 0.2340042999999996, 7.03132576]
[-0.4065281299999999, -0.0034688800000002296, 1.49913805]
[11.90736407, -1.0095234500000005, 0.1947113199999997]
[-1.3465853200000009, 10.33618977, -0.5635349600000001]
[0.6320950500000002, 1.36105104, -0.7944327800000002]
[0.19670237999999962, -0.06967930000000067, -1.6281831699999998]
[0.2052828299999998, -0.04091034999999987, -7.0397470900000005]
[-0.34020671000000036, -0.22818188000000017, -1.5953351800000002]
[-1.49662761, 0.1805147499999995, 0.30407385]
[1.48523666, -0.3292398799999998, -0.07847426000000013]
[-0.04928320999999958, 0.24726768, 1.5677787100000007]
[1.60166742, 1.1383598, 0.6881460500000003]
[-0.5291841800000006, 10.27195692, 0.17956839999999996]
[1.66629089, 0.1494993400

the following code:
import math

def find_angle(v1, v2):
    dot_product = sum(a * b for a, b in zip(v1, v2))
    magnitude_v1 = math.sqrt(sum(a**2 for a in v1))
    magnitude_v2 = math.sqrt(sum(b**2 for b in v2))
    theta = math.acos(dot_product / (magnitude_v1 * magnitude_v2))
    
    return math.degrees(theta)

calculates the angle between two vectors, I have two text files, that contain n rows of vectors, find the angle between the row 1 of first text file and row 1 of the second text file until the end which is the angle between the nth row of first text file and the nth row of the second text file. 
print the results in a text file named: ALL_ANGLS.txt

In [44]:
import math

def find_angle(v1, v2):
    dot_product = sum(a * b for a, b in zip(v1, v2))
    magnitude_v1 = math.sqrt(sum(a**2 for a in v1))
    magnitude_v2 = math.sqrt(sum(b**2 for b in v2))
    theta = math.acos(dot_product / (magnitude_v1 * magnitude_v2))
    
    return math.degrees(theta)

def read_vectors_from_file(filename):
    vectors = []
    with open(filename, 'r') as file:
        for line in file:
            vector = [float(elem) for elem in line.split()]
            vectors.append(vector)
    return vectors

file1 = "1_2.txt"
file2 = "3_2.txt"

vectors1 = read_vectors_from_file(file1)
vectors2 = read_vectors_from_file(file2)

if len(vectors1) != len(vectors2):
    print("Number of rows in both files should be equal.")

for i in range(min(len(vectors1), len(vectors2))):
    angle = find_angle(vectors1[i], vectors2[i])
    print(f"Angle between row {i+1} of {file1} and row {i+1} of {file2} is: {angle} degrees")

Angle between row 1 of 1_2.txt and row 1 of 3_2.txt is: 16.777723644376245 degrees
Angle between row 2 of 1_2.txt and row 2 of 3_2.txt is: 14.607223166071616 degrees
Angle between row 3 of 1_2.txt and row 3 of 3_2.txt is: 29.81080095844507 degrees
Angle between row 4 of 1_2.txt and row 4 of 3_2.txt is: 170.7765789512992 degrees
Angle between row 5 of 1_2.txt and row 5 of 3_2.txt is: 167.30541918874778 degrees
Angle between row 6 of 1_2.txt and row 6 of 3_2.txt is: 165.11825157334073 degrees
Angle between row 7 of 1_2.txt and row 7 of 3_2.txt is: 145.61043864440518 degrees
Angle between row 8 of 1_2.txt and row 8 of 3_2.txt is: 143.9093015890578 degrees
Angle between row 9 of 1_2.txt and row 9 of 3_2.txt is: 61.76502693684703 degrees
Angle between row 10 of 1_2.txt and row 10 of 3_2.txt is: 23.00465062347457 degrees
Angle between row 11 of 1_2.txt and row 11 of 3_2.txt is: 166.05103963399347 degrees
Angle between row 12 of 1_2.txt and row 12 of 3_2.txt is: 22.00765294070184 degrees
Angl

In [45]:
import math

def find_angle(v1, v2):
    dot_product = sum(a * b for a, b in zip(v1, v2))
    magnitude_v1 = math.sqrt(sum(a**2 for a in v1))
    magnitude_v2 = math.sqrt(sum(b**2 for b in v2))
    theta = math.acos(dot_product / (magnitude_v1 * magnitude_v2))
    
    return math.degrees(theta)

def read_vectors_from_file(filename):
    vectors = []
    with open(filename, 'r') as file:
        for line in file:
            vectors.append([float(x) for x in line.split()])
    return vectors

def calculate_all_angles(file1, file2):
    vectors1 = read_vectors_from_file(file1)
    vectors2 = read_vectors_from_file(file2)
    
    all_angles = []
    for v1, v2 in zip(vectors1, vectors2):
        all_angles.append(find_angle(v1, v2))
    
    with open('ALL_ANGLES.txt', 'w') as output_file:
        for angle in all_angles:
            output_file.write(str(angle) + '\n')

# Replace 'file1.txt' and 'file2.txt' with the actual filenames of your text files containing vectors
calculate_all_angles('1_2.txt', '3_2.txt')

# for the whole dataset

In [25]:
with open('all_converted_coordinates.txt', 'r') as file:
    lines = file.readlines()

# Define the functions to subtract two rows
def subtract_rows(row1, row2):
    return [row2[i] - row1[i] for i in range(len(row1))]

# Initialize a list to store the results
all_results = []

dataset = []
for line in lines:
    if line.startswith("Direct configuration"):
        if dataset:
            results = []
            for i in range(len(column1)):  # it iterates over all 197 Angles we have
                row_column1 = dataset[int(column1[i])]  # column1 and column2: These are predefined lists 
                row_column2 = dataset[int(column2[i])]  # of indices specifying which rows to subtract.
                
                     # For each index pair in column1 and column2, the corresponding rows from dataset are subtracted.
                result = subtract_rows(row_column1, row_column2)
                # Results are stored in results.
                results.append(result)
                # Appends the header (configuration line) and results to all_results.
            all_results.append((header, results))
        dataset = []  # Reset the dataset for the new set of data
        header = line  # Store the "Direct configuration" line
    else:
        dataset.append(list(map(float, line.split())))

# Write the results to a new text file
with open('1_2_all_results.txt', 'w') as file:
    for header, results in all_results:
        file.write(header)
        for result in results:
            file.write(' '.join(map(str, result)) + '\n')

print("Results have been written to all_results.txt file.")

Results have been written to all_results.txt file.


In [26]:
with open('all_converted_coordinates.txt', 'r') as file:
    lines = file.readlines()

# Define the functions to subtract two rows
def subtract_rows(row1, row2):
    return [row2[i] - row1[i] for i in range(len(row1))]

# Initialize a list to store the results
all_results = []

dataset = []
for line in lines:
    if line.startswith("Direct configuration"):
        if dataset:
            results = []
            for i in range(len(column1)):
                row_column2 = dataset[int(column2[i])]
                row_column3 = dataset[int(column3[i])]
                result = subtract_rows(row_column2, row_column3)
                results.append(result)
            all_results.append((header, results))
        dataset = []  # Reset the dataset for the new set of data
        header = line  # Store the "Direct configuration" line
    else:
        dataset.append(list(map(float, line.split())))

# Write the results to a new text file
with open('2_3_all_results.txt', 'w') as file:
    for header, results in all_results:
        file.write(header)
        for result in results:
            file.write(' '.join(map(str, result)) + '\n')

print("Results have been written to all_results.txt file.")

Results have been written to all_results.txt file.


In [27]:
import math

def find_angle(v1, v2):
    dot_product = sum(a * b for a, b in zip(v1, v2))
    magnitude_v1 = math.sqrt(sum(a**2 for a in v1))
    magnitude_v2 = math.sqrt(sum(b**2 for b in v2))
    theta = math.acos(dot_product / (magnitude_v1 * magnitude_v2))
    
    return math.degrees(theta)

def read_vectors_from_file(filename):
    vectors = []
    with open(filename, 'r') as file:
        for line in file:
            vectors.append([float(x) for x in line.split()])
    return vectors

def calculate_all_angles(input_file1, input_file2, output_filename):
    with open(input_file1, 'r') as file1, open(input_file2, 'r') as file2, open(output_filename, 'w') as output_file:
        lines1 = file1.readlines()
        lines2 = file2.readlines()
        
        header = None
        all_angles = []
        for line1, line2 in zip(lines1, lines2):
            if line1.startswith("Direct configuration") and line2.startswith("Direct configuration"):
                if header is not None:
                    output_file.write(header)
                    for angle in all_angles:
                        output_file.write(str(angle) + '\n')
                    all_angles = []
                header = line1
                
            else:
                vectors1 = [float(x) for x in line1.split()]
                vectors2 = [float(x) for x in line2.split()]
                angle = find_angle(vectors1, vectors2)
                all_angles.append(angle)
        
        if header is not None:
            output_file.write(header)
            for angle in all_angles:
                output_file.write(str(angle) + '\n')

# Replace '1_2_all_results.txt' and '2_3_all_results.txt' with the actual filenames
# of your text files containing the datasets
calculate_all_angles('1_2_all_results.txt', '2_3_all_results.txt', 'ALL_ANGLES.txt')

print("Angles for all datasets have been calculated and written to ALL_ANGLES.txt.")

Angles for all datasets have been calculated and written to ALL_ANGLES.txt.


 I have a text file (ALL_ANGLES.txt) with many rows, some lines start with "Direct" but except those lines I have other rows, for
 these rows write a python code that if the number is less than 90, subtract it from 180 otherwise keep it 
 untouched. print all results in a new text file named Final_Angles.txt and also in this file 
  print the same lines which start with "Direct" in the same positions as they are in the initial file. Thanks a lot honey.

In [28]:
# Open the input file for reading and the output file for writing
with open('ALL_ANGLES.txt', 'r') as infile, open('Final_Angles.txt', 'w') as outfile:
    for line in infile:
        # Check if the line starts with "Direct"
        if line.strip().startswith("Direct"):
            # Write the line as it is to the output file
            outfile.write(line)
        else:
            # Process numerical lines
            numbers = line.split()
            # Check if the line contains numbers
            if numbers and all(num.replace('.', '', 1).isdigit() for num in numbers):
                # Process each number in the line
                processed_numbers = [
                    str(180 - float(num)) if float(num) < 90 else num
                    for num in numbers
                ]
                # Write the processed line to the output file
                outfile.write(" ".join(processed_numbers) + "\n")
            else:
                # If the line does not contain valid numbers, write it as is
                outfile.write(line)


Check for correctness of Code

In [ ]:
# The first row of Output_Angles.txt is 67 4 75
# 4 is row number 6 (indixes begin at 0) and 67 is 69 ... 
7.52404187 10.18724811 4.29977104
7.54168492 10.49923346 2.70706037